# Limpieza del DataSet: IMDB 5000 Movie

En este notebook se realiza un proceso de limpieza de datos incluyendo:

1. Carga del dataset
2. Renombrar y ordenar columnas
3. Gestión de valores nulos
4. Corrección de tipos
	1. Corrección de tipos numéricos
	2. Normalización de strings
	3. Limpieza de columnas con valores múltiples
5. Detección y eliminación de duplicados
6. Guardar dataset

## 1. Carga del DataSet

In [464]:
import pandas as pd
import numpy as np
import re

In [465]:
df = pd.read_csv("movie_metadata.csv")
df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


## 2. Renombrar y ordenar columnas

In [466]:
df.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')

Las columnas están desordenadas y con nombres que a lo mejor no son aclarativos. Por eso se van a modificar las cabeceras para que sean mas leibles y descriptivas.

Esto facilitará la manipulación de datos más adelante.

In [ ]:
df.columns = [col.replace("_", " ").title() for col in df.columns]

nuevo_orden = [
    # Información básica de la película
    "Movie Title", "Title Year", "Movie Imdb Link", "Language", "Country", "Content Rating",
    # Información cinematográfica
    "Duration", "Color", "Genres", "Aspect Ratio", "Plot Keywords", "Facenumber In Poster",
    # Reparto
    "Director Name", "Actor 1 Name", "Actor 2 Name", "Actor 3 Name",
    # Dinero
    "Budget", "Gross",
    # Calidad
    "Num Voted Users", "Num User For Reviews", "Num Critic For Reviews", "Imdb Score",
    # Facebook
    "Director Facebook Likes", "Actor 1 Facebook Likes", "Actor 2 Facebook Likes",
    "Actor 3 Facebook Likes", "Cast Total Facebook Likes", "Movie Facebook Likes"
]

df = df[nuevo_orden]

## 3. Revisión de nulos y tipos

In [468]:
df.isna().sum()

Movie Title                    0
Title Year                   108
Movie Imdb Link                0
Language                      14
Country                        5
Content Rating               303
Duration                      15
Color                         19
Genres                         0
Aspect Ratio                 329
Plot Keywords                153
Facenumber In Poster          13
Director Name                104
Actor 1 Name                   7
Actor 2 Name                  13
Actor 3 Name                  23
Budget                       492
Gross                        884
Num Voted Users                0
Num User For Reviews          21
Num Critic For Reviews        50
Imdb Score                     0
Director Facebook Likes      104
Actor 1 Facebook Likes         7
Actor 2 Facebook Likes        13
Actor 3 Facebook Likes        23
Cast Total Facebook Likes      0
Movie Facebook Likes           0
dtype: int64

Viendo la cantidad de valores nulos de cada columna se pueden dividir en 3 grupos:
- Columnas sin nulos: Columnas que no contienen nulos y por lo tanto no son importantes a observar en este apartado.
- Columnas esenciales con nulos: Columnas que al tener nulos hacen que haya que eliminar la fila.
- Columnas no esenciales con nulos: Columnas en las que los nulos se les puede dar un valor por defecto.

Desglose:
- Sin nulos: Titulo, URL IMDB, Generos, Votos usuarios IMDB, Puntuacion IMDB, Facebook likes del reparto, Facebook likes pelicula.
- Esenciales: Año lanzamiento, Duracion, Nombre director, Nombre actor 1.
- No esenciales: Idiomas, Pais, Clasificacion, Color, Relacion aspecto, Palabras clave trama, Caras en poster, Nombre actor 2, Nombre actor 3, Presupuesto, Recaudacion, Numero reseñas usuarios, Numero reseñas criticos, Facebook likes director, Facebook likes actor 1, Facebook likes actor 2, Facebook likes actor 3.

In [469]:
# Número de filas actualmente en el dataset
len(df)

5043

In [470]:
# ESENCIALES:
esenciales = ["Title Year", "Duration", "Director Name", "Actor 1 Name"]
df = df.dropna(subset=esenciales)

# Número de filas tras eliminar las filas con valores nulos en columnas esenciales
len(df)

4916

Se pasa de tener 5043 filas a 4916 tras eliminar todas las filas con nulos en las columnas consideradas esenciales.

In [471]:
# NO ESENCIALES:
for col in ["Language", "Country", "Content Rating", "Color", "Plot Keywords", 
            "Actor 2 Name", "Actor 3 Name"]:
    df[col] = df[col].fillna("Unknown")
    
for col in ["Facenumber In Poster", "Num User For Reviews", "Num Critic For Reviews", 
            "Director Facebook Likes", "Actor 1 Facebook Likes", 
            "Actor 2 Facebook Likes", "Actor 3 Facebook Likes"]:
    df[col] = df[col].fillna(0)
    
for col in ["Aspect Ratio", "Budget", "Gross"]:
    df[col] = df[col].fillna(df[col].median())

df.isna().sum()

Movie Title                  0
Title Year                   0
Movie Imdb Link              0
Language                     0
Country                      0
Content Rating               0
Duration                     0
Color                        0
Genres                       0
Aspect Ratio                 0
Plot Keywords                0
Facenumber In Poster         0
Director Name                0
Actor 1 Name                 0
Actor 2 Name                 0
Actor 3 Name                 0
Budget                       0
Gross                        0
Num Voted Users              0
Num User For Reviews         0
Num Critic For Reviews       0
Imdb Score                   0
Director Facebook Likes      0
Actor 1 Facebook Likes       0
Actor 2 Facebook Likes       0
Actor 3 Facebook Likes       0
Cast Total Facebook Likes    0
Movie Facebook Likes         0
dtype: int64

Se puede comprobar que se han limpiado todos los datos tras rellenar los nulos de las columnas no esenciales con valores por defecto.

## 4. Corrección de tipos

### 4.1. Corrección de tipos numéricos

Se va a asegurar que todas las columnas numéricas sean del tipo correcto (`int` o `float`).
Esto permitirá operaciones matemáticas y estadísticas sin errores.

In [472]:
col_float = ["Aspect Ratio", "Budget", "Gross", "Imdb Score"]

for col in col_float:
    df[col] = df[col].astype(float)

In [473]:
col_int = ["Title Year", "Duration", "Facenumber In Poster", "Num Voted Users", "Num User For Reviews", "Num Critic For Reviews", "Director Facebook Likes", 
	"Actor 1 Facebook Likes", "Actor 2 Facebook Likes", "Actor 3 Facebook Likes", "Cast Total Facebook Likes", "Movie Facebook Likes"]

for col in col_int:
    df[col] = df[col].astype(int)

### 4.2. Normalización de strings

Se van a quitar los espacios al inicio y al final de los strings, y esto evita errores de coincidencia en filtros o análisis de categorías.

In [474]:
col_string = ["Movie Title", "Movie Imdb Link", "Language", "Country", "Content Rating", "Color",
              "Genres", "Plot Keywords", "Director Name", "Actor 1 Name",
              "Actor 2 Name", "Actor 3 Name", ]

for col in col_string:
    df[col] = df[col].astype(str).str.strip()

### 4.3. Limpieza de columnas con valores múltiples

Se van a limpiar los espacios y carácteres especiales (`\n`, `\t` y `"`) en "Genres" y "Plot Keywords" en cada unos de sus valores.

Antes de nada reseteamos el indice para poder recorrer correctamente el df, ya que al haber eliminado filas puede darnos error.

In [475]:
df = df = df.reset_index(drop=True)

In [476]:
# Genre
for i in range(len(df)):
    generos = df.loc[i, "Genres"]
    partes = generos.split("|")
    partes_limpias = [re.sub(r'[\n\t"]', '', g.strip()) for g in partes]
    df.loc[i, "Genres"] = "|".join(partes_limpias)

In [477]:
# Plot Keywords
for i in range(len(df)):    
    kw = df.loc[i, "Plot Keywords"]
    partes = kw.split("|")
    partes_limpias = [re.sub(r'[\n\t"]', '', k.strip()) for k in partes]
    df.loc[i, "Plot Keywords"] = "|".join(partes_limpias)

## 5. Detección y eliminación de duplicados

Se van a identificar y tratar si existen filas duplicadas.

En este caso se considera que ninguna de las columnas nombradas anteriormente actúan como id, ya que se puede repetir su valor en varias filas.

Por lo tanto, se van a buscar filas cuyos campos sean todos iguales (`fila1==fila2`)

In [478]:
duplicadas = df[df.duplicated(keep="first")]
duplicadas.sort_values(by="Movie Title")

,Movie Title,Title Year,Movie Imdb Link,Language,Country,Content Rating,Duration,Color,Genres,Aspect Ratio,...,Num Voted Users,Num User For Reviews,Num Critic For Reviews,Imdb Score,Director Facebook Likes,Actor 1 Facebook Likes,Actor 2 Facebook Likes,Actor 3 Facebook Likes,Cast Total Facebook Likes,Movie Facebook Likes
4826,A Dog's Breakfast,2007,http://www.imdb.com/title/tt0796314/?ref_=fn_t...,English,Canada,Unknown,88,Color,Comedy,1.78,...,3262,46,8,7.0,686,847,686,405,2364,377
4302,"A Woman, a Gun and a Noodle Shop",2009,http://www.imdb.com/title/tt1428556/?ref_=fn_t...,Mandarin,China,R,95,Color,Comedy|Drama,2.35,...,2410,20,101,5.7,611,9,4,3,18,784
2508,Bad Moms,2016,http://www.imdb.com/title/tt4651520/?ref_=fn_t...,English,USA,R,100,Color,Comedy,2.35,...,4654,46,81,6.7,24,15000,1000,851,18786,18000
2738,Big Fat Liar,2002,http://www.imdb.com/title/tt0265298/?ref_=fn_t...,English,USA,PG,88,Color,Adventure|Comedy|Family,1.85,...,29008,99,69,5.4,189,934,927,799,3707,896
4821,Cat People,1982,http://www.imdb.com/title/tt0083722/?ref_=fn_t...,English,USA,R,93,Color,Fantasy|Horror|Thriller,1.85,...,14193,106,130,6.1,261,783,782,697,3700,0
3380,Crash,2004,http://www.imdb.com/title/tt0375679/?ref_=fn_t...,English,Germany,R,115,Color,Crime|Drama|Thriller,2.35,...,361169,1624,287,7.9,549,3000,912,911,5732,18000
4653,Crossroads,2002,http://www.imdb.com/title/tt0275022/?ref_=fn_t...,English,USA,PG-13,93,Color,Comedy|Drama,1.85,...,34219,578,111,3.3,33,1000,188,135,1531,0
2717,Dangerous Liaisons,1988,http://www.imdb.com/title/tt0094947/?ref_=fn_t...,English,USA,R,119,Color,Drama|Romance,1.85,...,52846,143,51,7.7,350,18000,17000,418,35501,0
2245,Death at a Funeral,2007,http://www.imdb.com/title/tt0795368/?ref_=fn_t...,English,USA,R,87,Color,Comedy,1.85,...,89547,199,168,7.4,0,22000,557,548,24324,0
383,Fantastic Four,2015,http://www.imdb.com/title/tt1502712/?ref_=fn_t...,English,USA,PG-13,100,Color,Action|Adventure|Sci-Fi,2.35,...,110486,695,369,4.3,128,596,360,78,1261,41000


44 películas duplicadas que hay que eliminar.

In [479]:
df.drop_duplicates(inplace=True)

## 6. Guardar DataSet

In [480]:
df.to_csv("imdb_5000_movies_limpio.csv", index=False)